In [24]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/Documents/git_workspace/16715/dynamics-simulation-leg/scripts/Project.toml`


In [25]:
using RigidBodyDynamics
using LinearAlgebra
using MeshCatMechanisms
using MeshCat
using StaticArrays
using SparseArrays
using ForwardDiff

In [26]:
# link lengths
const l0 = [0.1; 0; 0]
const l1 = [0.3; 0; 0]
const l2 = [0.3; 0; 0]
const l3 = 0.1
const l4 = 0.2
const l5 = 0.0205
const lc = [l3; 0; 0]
const lee = [l3 + l4; 0; l5] # sqrt((l3 + l4)^2 + l5^2)

# CoM locations
const l_cb = [0; 0.004; 0]
const l_c0 = [0.0125108364230515; 0.00117191218927888; 0]
const l_c1 = [0.149359714867044; 0; 0]
const l_c2 = [0.0469412900551914; 0; 0]
const l_c3 = [0.113177000131857; 0; -0.015332867880069]

# link masses
const mb = 7  # kg
const m0 = 0.24644240965487
const m1 = 0.0707939028219395
const m2 = 0.276735496985514
const m3 = 0.130824780046739
# const m = Diagonal([m0, m1, m2, m3])
    
# gravity, obviously
const g = 9.807
const grav = [0; 0; g]
# mass moment of inertia in axis of rotation
const Ib = Array([0.0024241 5.252E-06 2.0733E-19; 
                  5.252E-06 0.0044176 -3.1153E-19; 
                  2.0733E-19 -3.1153E-19 0.0022481])

const I0 = Array([3.83120149546952E-05 1.46925714738609E-05 -8.60106401672571E-06;
                  1.46925714738609E-05 0.000172067745507247 1.0427260925207E-06;
                  -8.60106401672571E-06 1.0427260925207E-06 0.00014745218068435])

const I1 = Array([3.06999775886187E-06 7.91090301514898E-12 -1.43705963146176E-12;
                  7.91090301514898E-12 0.000147960574744097 1.30742394049546E-11;
                  -1.43705963146176E-12 1.30742394049546E-11 0.000147884231885009])

const I2 = Array([3.43038397803592E-05 -2.90339844227483E-07 6.18680397558952E-06;
                  -2.90339844227483E-07 0.000302324068012293 2.25016327583562E-08;
                  6.18680397558952E-06 2.25016327583562E-08 0.00028292376778719])

const I3 = Array([1.76996970020568E-05 -5.3695427116208E-07 7.62350214406387E-07;
                  -5.3695427116208E-07 0.000164188445564489 -2.77843753828047E-07;
                  7.62350214406387E-07 -2.77843753828047E-07 0.000160656046697151])

3×3 Matrix{Float64}:
  1.76997e-5  -5.36954e-7    7.6235e-7
 -5.36954e-7   0.000164188  -2.77844e-7
  7.6235e-7   -2.77844e-7    0.000160656

In [27]:
M̄ = [mb*I(3) zeros(3, 27)
     zeros(3,3) Ib zeros(3, 24)
     zeros(3,6) m0*I(3) zeros(3,21);
     zeros(3,9) I0 zeros(3,18);
     zeros(3,12) m1*I(3) zeros(3,15);
     zeros(3,15) I1 zeros(3, 12);
     zeros(3,18) m2*I(3) zeros(3, 9);
     zeros(3, 21) I2 zeros(3, 6);
     zeros(3, 24) m3*I(3) zeros(3, 3);
     zeros(3, 27) I3]

30×30 SparseMatrixCSC{Float64, Int64} with 60 stored entries:
⠑⢄⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠘⠛⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⢱⣶⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠑⢄⣀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠘⠛⢄⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢱⣶⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⣀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠛

In [28]:
function hat(ω)
    return [0 -ω[3] ω[2];
            ω[3] 0 -ω[1];
            -ω[2] ω[1] 0]
end

function L(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I + hat(Q[2:4])]
end

function R(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I - hat(Q[2:4])]
end

H = [zeros(1,3); I];

T = Diagonal([1.0; -1; -1; -1])

function G(Q)
    return L(Q)*H  # 4x3
end

function Ḡ(q)
    Qb = q[4:7]
    Q0 = q[11:14]
    Q1 = q[18:21]
    Q2 = q[25:28]
    Q3 = q[32:35]
    
    # [I(3) zeros(3,4); zeros(4,3) G(Q) zeros(4,1); zeros(1,6) 1]  # 8x7
    return blockdiag(sparse(I, 3, 3), sparse(G(Qb)), 
                     sparse(I, 3, 3), sparse(G(Q0)), 
                     sparse(I, 3, 3), sparse(G(Q1)),
                     sparse(I, 3, 3), sparse(G(Q2)),
                     sparse(I, 3, 3), sparse(G(Q3)),)
end

Ḡ (generic function with 1 method)

In [29]:
function Expq(ϕ)
    
    # The quaternion exponential map ϕ → q 
    q = zeros(4)
    θ = norm(ϕ)
    q = [cos(θ/2); 0.5*ϕ*sinc(θ/(2*pi))]
    
    return q
end

function rotate(Q, p)
    # Rotate a position vector p by a quaternion Q
    return H'L(Q)*R(Q)'*H*p
end

rotate (generic function with 1 method)

In [31]:
function del(m, I, r1, r2, r3, Q1, Q2, Q3)
    [(1/h)*m*(r2-r1) - (1/h)*m*(r3-r2) - m*grav*h;
     (2.0/h)*G(Q2)'*L(Q1)*H*I*H'*L(Q1)'*Q2 + (2.0/h)*G(Q2)'*T*R(Q3)'*H*I*H'*L(Q2)'*Q3]
end
    
    
function DEL(q_1,q_2,q_3,λ,F1,F2)
    
    rb_1 = q_1[1:3]
    Qb_1 = q_1[4:7]
    r0_1 = q_1[8:10]
    Q0_1 = q_1[11:14]
    r1_1 = q_1[15:17]
    Q1_1 = q_1[18:21]
    r2_1 = q_1[22:24]
    Q2_1 = q_1[25:28]
    r3_1 = q_1[29:31]
    Q3_1 = q_1[32:35]
    
    rb_2 = q_2[1:3]
    Qb_2 = q_2[4:7]
    r0_2 = q_2[8:10]
    Q0_2 = q_2[11:14]
    r1_2 = q_2[15:17]
    Q1_2 = q_2[18:21]
    r2_2 = q_2[22:24]
    Q2_2 = q_2[25:28]
    r3_2 = q_2[29:31]
    Q3_2 = q_2[32:35]
    
    rb_3 = q_3[1:3]
    Qb_3 = q_3[4:7]
    r0_3 = q_3[8:10]
    Q0_3 = q_3[11:14]
    r1_3 = q_3[15:17]
    Q1_3 = q_3[18:21]
    r2_3 = q_3[22:24]
    Q2_3 = q_3[25:28]
    r3_3 = q_3[29:31]
    Q3_3 = q_3[32:35]

    del1 = [del(mb, Ib, rb_1, rb_2, rb_3, Qb_1, Qb_2, Qb_3);
            del(m0, I0, r0_1, r0_2, r0_3, Q0_1, Q0_2, Q0_3);
            del(m1, I1, r1_1, r1_2, r1_3, Q1_1, Q1_2, Q1_3);
            del(m2, I2, r2_1, r2_2, r2_3, Q2_1, Q2_2, Q2_3);
            del(m3, I3, r3_1, r3_2, r3_3, Q3_1, Q3_2, Q3_3)] 

    return del1 + (h/2.0)*F1 + (h/2.0)*F2 + h*Dc(q_2)'*λ
end

DEL (generic function with 1 method)

In [32]:
function Dq3DEL(q_1,q_2,q_3,λ,F1,F2)
    # @show Ḡ(q_3)
    ForwardDiff.jacobian(dq->DEL(q_1,q_2,dq,λ,F1,F2), q_3)*Ḡ(q_3)
end

Dq3DEL (generic function with 1 method)

In [33]:
function c(q)
    # constraint function
    # c(q_0) should be all zeros
    rb = q[1:3]
    Qb = q[4:7]
    r0 = q[8:10]
    Q0 = q[11:14]
    r1 = q[15:17]
    Q1 = q[18:21]
    r2 = q[22:24]
    Q2 = q[25:28]
    r3 = q[29:31]
    Q3 = q[32:35]
    
    pb = rb + rotate(Qb, l_cb) # position vector from world frame to *JOINTS* 0 and 2
    
    c_ = [pb - r0 - rotate(Q0, -l_c0);  # ???
         [0 1 0 0; 0 0 0 1]*L(Qb)'*Q0;  # y axis rotation constraint
          r0 + rotate(Q0,  l0-l_c0) - r1 - rotate(Q1, -l_c1);
         [0 1 0 0; 0 0 0 1]*L(Q0)'*Q1;
          pb - r2 - rotate(Q2, -l_c2);
         [0 1 0 0; 0 0 0 1]*L(Qb)'*Q2;
          r2 + rotate(Q2, l2 - l_c2) - r3 - rotate(Q3, -l_c3);
         [0 1 0 0; 0 0 0 1]*L(Q2)'*Q3;
          r1 + rotate(Q1, l1 - l_c1) - r3 - rotate(Q3, lc-l_c3)]
         #[0 1 0 0; 0 0 0 1]*L(Q1)'*Q3]
end

c (generic function with 1 method)

In [34]:
function Dc(q)
    ForwardDiff.jacobian(dq->c(dq),q)*Ḡ(q)
end

Dc (generic function with 1 method)

In [50]:
#Initial Conditions
rb = zeros(3)
Qb = [1.0; 0; 0; 0]
q0 = 30*pi/180;
q1 = 120*(pi/180)
q2 = 150*(pi/180)
q3 = -120*(pi/180)

Tf = 10.0
h = 0.001 #20 Hz
thist = 0:h:Tf
N = length(thist)

pb = rb + rotate(Qb, l_cb)  # position vector from world frame to *JOINTS* 0 and 2

Qb0 = Expq([0, q0, 0])  # quaternion from base to link 0
Q0 = L(Qb)*Qb0  # quaternion from world frame to link 0
r0 = pb + rotate(Q0, l_c0)

Q01 = Expq([0, q1, 0])  # quaternion from link 0 to link 1
Q1 = L(Q0)*Q01  # quaternion from world frame to link 1
r1 = pb + rotate(Q0, l0) + rotate(Q1, l_c1)  

Qb2 = Expq([0, q2, 0])  # quaternion from base to link 2
Q2 = L(Qb)*Qb2  # quaternion from world frame to link 2
r2 = pb + rotate(Q2, l_c2)

Q23 = Expq([0, q3, 0])  # quaternion from base to link 2
Q3 = L(Q2)*Q23  # quaternion from world frame to link 2
r3 = pb + rotate(Q2, l2) + rotate(Q3, l_c3)  

rb_0 = rb
r0_0 = r0
r1_0 = r1
r2_0 = r2
r3_0 = r3
Qb_0 = Qb
Q0_0 = Q0
Q1_0 = Q1
Q2_0 = Q2
Q3_0 = Q3

q_0 = [rb_0; Qb_0; r0_0; Q0_0; r1_0; Q1_0; r2_0; Q2_0; r3_0; Q3_0]
# @show c(q_0)


35-element Vector{Float64}:
  0.0
  0.0
  0.0
  1.0
  0.0
  0.0
  0.0
  0.010834702164954238
  0.00517191218927888
 -0.006255418211525749
  0.9659258262890683
  0.0
  0.25881904510252074
  ⋮
 -0.023470645027595695
  0.25881904510252074
  0.0
  0.9659258262890683
  0.0
 -0.16945989783706322
  0.004
 -0.2198671531629387
  0.9659258262890683
  0.0
  0.2588190451025209
  0.0

In [51]:
#Torque input at joints
uhist = repeat([0 0 0 0 0]*1e-4, N)'
@show size(uhist)
# 1 -> joint0
# 2 -> joint1
# 3 -> joint2
# 4 -> joint3
# 5 -> joint4 (parallel constraint)
#Corresponding F
Fhist = zeros(30,N)  # wrench [xyz force, xyz torque]
for k = 1:N
    Fhist[:,k] = [zeros(4); -uhist[1, k]-uhist[3, k]; 0; # body
                  zeros(4); uhist[1, k]-uhist[2, k]; 0; # link0
                  zeros(4); uhist[2, k]+uhist[5, k]; 0; # link1
                  zeros(4); uhist[3, k]-uhist[4, k]; 0; # link2
                  zeros(4); uhist[4, k]-uhist[5, k]; 0] # link3
end

size(uhist) = (5, 10001)


In [53]:
#Simulate
qhist = zeros(35,N)
qhist[:,1] .= q_0
qhist[:,2] .= q_0

@show n_c = size(c(q_0))[1]  # number of constraint rows
flush(stdout)
for k = 2:(N-1)
    
    #Initial guess
    qhist[:,k+1] .= qhist[:,k]
    λ = zeros(n_c)
    
    e = [DEL(qhist[:,k-1],qhist[:,k],qhist[:,k+1],λ,Fhist[:,k-1],Fhist[:,k]); c(qhist[:,k+1])]

    while maximum(abs.(e)) > 1e-8
        D = Dq3DEL(qhist[:,k-1],qhist[:,k],qhist[:,k+1],λ,Fhist[:,k-1],Fhist[:,k])
        C2 = Dc(qhist[:,k])
        C3 = Dc(qhist[:,k+1])

        Δ = -([D h*C2'; C3 zeros(n_c,n_c)]\e)
        
        qhist[1:3,k+1] .= qhist[1:3,k+1] + Δ[1:3]
        qhist[4:7,k+1] .= L(qhist[4:7,k+1])*[sqrt(1-Δ[4:6]'*Δ[4:6]); Δ[4:6]]
        qhist[8:10,k+1] .= qhist[8:10,k+1] + Δ[7:9]
        qhist[11:14,k+1] .= L(qhist[11:14,k+1])*[sqrt(1-Δ[10:12]'*Δ[10:12]); Δ[10:12]]
        qhist[15:17,k+1] .= qhist[15:17,k+1] + Δ[13:15]
        qhist[18:21,k+1] .= L(qhist[18:21,k+1])*[sqrt(1-Δ[16:18]'*Δ[16:18]); Δ[16:18]]
        qhist[22:24,k+1] .= qhist[22:24,k+1] + Δ[19:21]
        qhist[25:28,k+1] .= L(qhist[25:28,k+1])*[sqrt(1-Δ[22:24]'*Δ[22:24]); Δ[22:24]]
        qhist[29:31,k+1] .= qhist[29:31,k+1] + Δ[25:27]
        qhist[32:35,k+1] .= L(qhist[32:35,k+1])*[sqrt(1-Δ[28:30]'*Δ[28:30]); Δ[28:30]]

        λ .= λ + Δ[31:30+n_c]
        
        e = [DEL(qhist[:,k-1],qhist[:,k],qhist[:,k+1],λ,Fhist[:,k-1],Fhist[:,k]); c(qhist[:,k+1])]
        # print(maximum(abs.(e)), "\n") # 
        # print(c(qhist[:,k+1]), "\n")
        # flush(stdout)
    end
    print("\r Simulation ", trunc(Int, k/(N-1)*100), "% complete")
    flush(stdout)
end

n_c = (size(c(q_0)))[1] = 23
 Simulation 100% completecomplete% complete% complete% complete% complete% complete% complete% complete% complete% complete% complete

In [54]:
qhist

35×10001 Matrix{Float64}:
  0.0          0.0          5.60678e-11  …     0.00283259     0.00283315
  0.0          0.0         -9.50487e-13       -5.84109e-5    -5.84239e-5
  0.0          0.0         -9.80696e-6      -490.201       -490.299
  1.0          1.0          1.0                0.844159       0.844098
  0.0          0.0         -6.42461e-11       -0.0076886     -0.00769064
  0.0          0.0         -1.13183e-8   …    -0.536017      -0.536112
  0.0          0.0         -1.95483e-10       -0.00480389    -0.00480371
  0.0108347    0.0108347    0.0108347          0.00254634     0.00254374
  0.00517191   0.00517191   0.00517191         0.00502078     0.00502079
 -0.00625542  -0.00625542  -0.00626523      -490.214       -490.312
  0.965926     0.965926     0.965926     …     0.696541       0.69645
  0.0          0.0         -1.14623e-11        0.00313928     0.00314073
  0.258819     0.258819     0.258819           0.717459       0.717548
  ⋮                                      ⋱  

In [55]:
using MeshCat, GeometryBasics, CoordinateTransformations, Rotations#, GeometryTypes
#const gt = GeometryTypes

In [56]:
vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8703
└ @ MeshCat /home/ben/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8703))

In [63]:
# Zoom in--the robot falls through floor since there is no contact

delete!(vis)

green_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.8))
red_material = MeshPhongMaterial(color=RGBA(1, 0, 0, 0.8))

#l_345 = sqrt((l3 + l4)^2 + l5^2)
# sphere = Sphere(Point(0, 0, 0), 3)
# rect = Rectangle(0.2, 0.2, 0.2, 0.2)
cylinderb = Cylinder(Point(0, -0.02, l_cb[2]), Point(0, 0.02, l_cb[2]), 0.04)  #float(l_cb)
cylinder0 = Cylinder(Point(-l_c0[1], -l_c0[2], -l_c0[3]), Point((l0-l_c0)[1], (l0-l_c0)[2], (l0-l_c0)[3]), 0.008)
cylinder1 = Cylinder(Point(-l_c1[1], -l_c1[2], -l_c1[3]), Point((l1-l_c1)[1],(l1-l_c1)[2],(l1-l_c1)[2]), 0.008)
cylinder2 = Cylinder(Point(-l_c2[1], -l_c2[2], -l_c2[3]), Point((l2-l_c2)[1], (l2-l_c2)[2], (l2-l_c2)[3]), 0.008)
cylinder3 = Cylinder(Point(-l_c3[1], -l_c3[2], -l_c3[3]), Point((lee-l_c3)[1], (lee-l_c3)[2],  (lee-l_c3)[3]), 0.008)

# setobject!(vis["sphere"],sphere,red_material)
setobject!(vis["cylinderb"],cylinderb,red_material)
setobject!(vis["cylinder0"],cylinder0,green_material)
setobject!(vis["cylinder1"],cylinder1,green_material)
setobject!(vis["cylinder2"],cylinder2,green_material)
setobject!(vis["cylinder3"],cylinder3,green_material)

for k = 1:N
    
    q = qhist[:, k]
    # set position and attitude
    
    positionb = Translation(q[1:3]...)
    attitudeb = LinearMap(UnitQuaternion(q[4:7]))
    position0 = Translation(q[8:10]...)
    attitude0 = LinearMap(UnitQuaternion(q[11:14]))
    position1 = Translation(q[15:17]...)
    attitude1 = LinearMap(UnitQuaternion(q[18:21]))
    position2 = Translation(q[22:24]...)
    attitude2 = LinearMap(UnitQuaternion(q[25:28]))
    position3 = Translation(q[29:31]...)
    attitude3 = LinearMap(UnitQuaternion(q[32:35]))

    settransform!(vis["cylinderb"], compose(positionb,attitudeb))
    settransform!(vis["cylinder0"], compose(position0,attitude0))
    settransform!(vis["cylinder1"], compose(position1,attitude1))
    settransform!(vis["cylinder2"], compose(position2,attitude2))
    settransform!(vis["cylinder3"], compose(position3,attitude3))
    #sleep(0.001)
end